# Analisis Model Keuangan: Beneish M-Score & Altman Z-Score

Notebook ini mendemonstrasikan penggunaan modul `FinancialScoreModels` untuk menghitung:
1.  **Beneish M-Score**: Untuk mendeteksi potensi manipulasi laba.
2.  **Altman Z-Score**: Untuk memprediksi risiko kebangkrutan perusahaan.

Data input diambil dari hasil ekstraksi `SaranaModule` yang disimpan dalam format JSON.

In [1]:
import json
from FinancialScoreModels.utils import load_sarana_output, get_company_financial_data
from FinancialScoreModels.beneish_m_score import get_beneish_m_score_analysis
from FinancialScoreModels.altman_z_score import get_altman_z_score_analysis

# --- Konfigurasi Path File ---
# Ganti dengan path yang sesuai jika output Sarana disimpan di lokasi berbeda
FILE_PATH_T = "Output/Sarana/hasil_ekstraksi_semua_dokumen.json"
# FILE_PATH_T_MINUS_1 = "Output/Sarana/hasil_ekstraksi_semua_dokumen_t_minus_1.json" # Komentari atau hapus untuk tes satu file
FILE_PATH_T_MINUS_1 = None # Atau set ke FILE_PATH_T jika ingin eksplisit dari file yang sama

# Nama file perusahaan target (harus cocok dengan 'nama_file' di JSON Sarana)
TARGET_COMPANY_FILENAME = "astra_lapkeu.pdf" 
# Untuk Altman Z-Score, tentukan apakah perusahaan publik atau tidak
# Jika publik dan Nilai Pasar Ekuitas diketahui, masukkan di market_value_equity_manual
IS_PUBLIC_COMPANY = True # Asumsi perusahaan publik untuk contoh ini
MARKET_VALUE_EQUITY_MANUAL = None # Contoh: 120000000000000.0 (jika diketahui, jika tidak, akan pakai nilai buku untuk X4 atau X4 jadi None)

## 1. Memuat Data Hasil Ekstraksi Sarana

In [2]:
try:
    data_t_list, data_t_minus_1_list = load_sarana_output(FILE_PATH_T, FILE_PATH_T_MINUS_1)
    print(f"Berhasil memuat data untuk periode t dari: {FILE_PATH_T}")
    if data_t_minus_1_list:
        if FILE_PATH_T_MINUS_1 and FILE_PATH_T_MINUS_1 != FILE_PATH_T:
            print(f"Berhasil memuat data untuk periode t-1 dari file spesifik: {FILE_PATH_T_MINUS_1}")
        else:
            print(f"Berhasil memuat data untuk periode t-1 dari file utama: {FILE_PATH_T}")
    else:
        print(f"Data untuk periode t-1 tidak berhasil dimuat atau tidak ditemukan.")
except Exception as e:
    print(f"Gagal memuat data: {e}")
    data_t_list = []
    data_t_minus_1_list = []

Berhasil memuat data untuk periode t dari: Output/Sarana/hasil_ekstraksi_semua_dokumen.json
Data untuk periode t-1 tidak berhasil dimuat atau tidak ditemukan.


## 2. Mengambil Data Keuangan Perusahaan Target

In [3]:
company_data_t = None
company_data_t_minus_1 = None

if data_t_list:
    company_data_t = get_company_financial_data(data_t_list, TARGET_COMPANY_FILENAME)
    if company_data_t:
        print(f"\nData keuangan periode t untuk {TARGET_COMPANY_FILENAME} ditemukan.")
        # print(json.dumps(company_data_t, indent=2))
    else:
        print(f"\nData keuangan periode t untuk {TARGET_COMPANY_FILENAME} TIDAK ditemukan.")

if data_t_minus_1_list:
    company_data_t_minus_1 = get_company_financial_data(data_t_minus_1_list, TARGET_COMPANY_FILENAME)
    if company_data_t_minus_1:
        print(f"Data keuangan periode t-1 untuk {TARGET_COMPANY_FILENAME} ditemukan.")
        # print(json.dumps(company_data_t_minus_1, indent=2))
    else:
        print(f"Data keuangan periode t-1 untuk {TARGET_COMPANY_FILENAME} TIDAK ditemukan.")


Data keuangan periode t untuk astra_lapkeu.pdf ditemukan.


## 3. Analisis Beneish M-Score

In [4]:
print("\n--- Analisis Beneish M-Score ---")
if company_data_t and company_data_t_minus_1:
    beneish_analysis = get_beneish_m_score_analysis(company_data_t, company_data_t_minus_1)
    print(f"Variabel Beneish untuk {TARGET_COMPANY_FILENAME}:")
    for var, value in beneish_analysis.get("beneish_variables", {}).items():
        print(f"  {var}: {value:.4f}" if value is not None else f"  {var}: N/A")
    m_score = beneish_analysis.get('m_score')
    print(f"Skor M: {m_score:.4f}" if m_score is not None else "Skor M: N/A")
    print(f"Interpretasi: {beneish_analysis.get('interpretation')}")
else:
    print("Tidak dapat melakukan analisis Beneish M-Score karena data periode t atau t-1 tidak lengkap.")


--- Analisis Beneish M-Score ---
Tidak dapat melakukan analisis Beneish M-Score karena data periode t atau t-1 tidak lengkap.


## 4. Analisis Altman Z-Score

In [5]:
print("\n--- Analisis Altman Z-Score ---")
if company_data_t:
    # Analisis sebagai perusahaan publik (default)
    print(f"\nAnalisis Altman Z-Score untuk {TARGET_COMPANY_FILENAME} (Asumsi: Perusahaan Publik)")
    altman_public_analysis = get_altman_z_score_analysis(company_data_t, is_public_company=True, market_value_equity_manual=MARKET_VALUE_EQUITY_MANUAL)
    print(f"Model: {altman_public_analysis.get('model_type')}")
    print(f"Variabel Altman untuk {TARGET_COMPANY_FILENAME}:")
    for var, value in altman_public_analysis.get("altman_variables", {}).items():
        print(f"  {var}: {value:.4f}" if value is not None else f"  {var}: N/A")
    z_score_public = altman_public_analysis.get('z_score')
    print(f"Skor Z: {z_score_public:.4f}" if z_score_public is not None else "Skor Z: N/A")
    print(f"Interpretasi: {altman_public_analysis.get('interpretation')}")
    
    # Analisis sebagai perusahaan non-publik (privat)
    print(f"\nAnalisis Altman Z-Score untuk {TARGET_COMPANY_FILENAME} (Asumsi: Perusahaan Non-Publik)")
    altman_private_analysis = get_altman_z_score_analysis(company_data_t, is_public_company=False)
    print(f"Model: {altman_private_analysis.get('model_type')}")
    print(f"Variabel Altman untuk {TARGET_COMPANY_FILENAME}:")
    for var, value in altman_private_analysis.get("altman_variables", {}).items():
        print(f"  {var}: {value:.4f}" if value is not None else f"  {var}: N/A")
    z_score_private = altman_private_analysis.get('z_score')
    print(f"Skor Z: {z_score_private:.4f}" if z_score_private is not None else "Skor Z': N/A")
    print(f"Interpretasi: {altman_private_analysis.get('interpretation')}")
else:
    print("Tidak dapat melakukan analisis Altman Z-Score karena data periode t tidak lengkap.")


--- Analisis Altman Z-Score ---

Analisis Altman Z-Score untuk astra_lapkeu.pdf (Asumsi: Perusahaan Publik)
Model: Publik Manufaktur
Variabel Altman untuk astra_lapkeu.pdf:
  X1: N/A
  X2: N/A
  X3: N/A
  X4: N/A
  X5: N/A
Skor Z: N/A
Interpretasi: Tidak dapat dihitung (data tidak lengkap atau X4 Market Value tidak ada untuk publik).

Analisis Altman Z-Score untuk astra_lapkeu.pdf (Asumsi: Perusahaan Non-Publik)
Model: Privat Manufaktur
Variabel Altman untuk astra_lapkeu.pdf:
  X1: N/A
  X2: N/A
  X3: N/A
  X4: N/A
  X5: N/A
Skor Z': N/A
Interpretasi: Tidak dapat dihitung (data tidak lengkap atau X4 Market Value tidak ada untuk publik).
